In [2]:
import pandas as pd

db = pd.read_csv("../lapacke_db/lapacke.csv", index_col=0)
db = db[~ db['arg_type'].map(lambda x: ('LAPACK' in x))]

print(db)

             fct_name return_type  e_name  arg_pos  \
0      LAPACKE_sbdsdc  lapack_int  sbdsdc        0   
1      LAPACKE_sbdsdc  lapack_int  sbdsdc        1   
2      LAPACKE_sbdsdc  lapack_int  sbdsdc        2   
3      LAPACKE_sbdsdc  lapack_int  sbdsdc        3   
4      LAPACKE_sbdsdc  lapack_int  sbdsdc        4   
...               ...         ...     ...      ...   
10406  LAPACKE_zupmtr  lapack_int  zupmtr        5   
10407  LAPACKE_zupmtr  lapack_int  zupmtr        6   
10408  LAPACKE_zupmtr  lapack_int  zupmtr        7   
10409  LAPACKE_zupmtr  lapack_int  zupmtr        8   
10410  LAPACKE_zupmtr  lapack_int  zupmtr        9   

                           arg_type       arg_name  
0                               int  matrix_layout  
1                              char           uplo  
2                              char          compq  
3                        lapack_int              n  
4                            float*              d  
...                             .